## Import the libraries

In [1]:
#importing the libraries
import tensorflow as tf  
from tensorflow import keras
import numpy as np
import pandas as pd

## Load the data and data pre-processing

In [2]:
#load the datasets
baseline_df = pd.read_excel('extracted_features_baseline.xlsx')
toolwear_df = pd.read_excel('extracted_features_toolwear.xlsx')

In [3]:
#labelling the datasets. 0 for baseline, 1 for toolwear. This will be the variable the model tries to predict
baseline_df["state"] = 0
toolwear_df["state"] = 1

In [4]:
#concantanate the datasets
combined_df = pd.concat([baseline_df, toolwear_df], axis=0)
print(combined_df.shape)

(840, 67)


In [5]:
#getting the y label 
state = combined_df["state"].values
print(state.shape)

(840,)


In [6]:
#getting the features to train the model
features = combined_df.drop('state', axis=1).values
print(features.shape)

(840, 66)


In [7]:
#train test split
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(features, state, test_size=0.2, random_state=50)

In [8]:
#data scalling
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Specify the architecture of the stacked autoencoder

In [9]:
#specify the number of condensed features for the 3 autoencoders. This will be the number of neurons in the hidden layer
condensed_e1 = 50
condensed_e2 = 30
condensed_e3 = 10

## Construction and training of the first-Level autoencoder

In [10]:
#constructing the first Autoencoder

#input layer 
in_ae1 = keras.Input(features.shape[1])
#hidden encoded layer
h_ae1 = keras.layers.Dense(condensed_e1)(in_ae1)
#output layer
out_ae1 = keras.layers.Dense(features.shape[1])(h_ae1)

ae1 = keras.Model(in_ae1, out_ae1)
encoder1 = keras.Model(in_ae1, h_ae1)

ae1.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 66)]              0         
                                                                 
 dense (Dense)               (None, 50)                3350      
                                                                 
 dense_1 (Dense)             (None, 66)                3366      
                                                                 
Total params: 6716 (26.23 KB)
Trainable params: 6716 (26.23 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [11]:
#compilling and training the first autoencoder
ae1.compile(optimizer='adam', loss='mse')
ae1.fit(X_train, X_train, epochs = 50, batch_size = 8, validation_split = 0.1)


Epoch 1/50

76/76 [==============================] - 1s 4ms/step - loss: 0.9498 - val_loss: 0.4013
Epoch 2/50
76/76 [==============================] - 0s 2ms/step - loss: 0.3305 - val_loss: 0.2230
Epoch 3/50
76/76 [==============================] - 0s 2ms/step - loss: 0.1962 - val_loss: 0.1463
Epoch 4/50
76/76 [==============================] - 0s 2ms/step - loss: 0.1315 - val_loss: 0.1025
Epoch 5/50
76/76 [==============================] - 0s 2ms/step - loss: 0.0950 - val_loss: 0.0772
Epoch 6/50
76/76 [==============================] - 0s 2ms/step - loss: 0.0734 - val_loss: 0.0611
Epoch 7/50
76/76 [==============================] - 0s 2ms/step - loss: 0.0597 - val_loss: 0.0511
Epoch 8/50
76/76 [==============================] - 0s 2ms/step - loss: 0.0502 - val_loss: 0.0430
Epoch 9/50
76/76 [==============================] - 0s 2ms/step - loss: 0.0431 - val_loss: 0.0376
Epoch 10/50
76/76 [==============================] - 0s 2ms/step - loss: 0.0374 - val_loss: 0.0330
Epoch 11/50
76/76

## Encoding the original features with the first-level encoder

In [12]:
#obtaining the first level of encoded features using the first encoder
encoded1 = encoder1.predict(X_train)

21/21 [==============================] - 0s 1ms/step


## Construction and training of the second-level autoencoder

In [13]:
#constructing the second Autoencoder

#input layer 
in_ae2 = keras.Input(condensed_e1)
#hidden encoded layer
h_ae2 = keras.layers.Dense(condensed_e2)(in_ae2)
#output layer
out_ae2 = keras.layers.Dense(condensed_e1)(h_ae2)

ae2 = keras.Model(in_ae2, out_ae2)
encoder2 = keras.Model(in_ae2, h_ae2)

ae2.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 50)]              0         
                                                                 
 dense_2 (Dense)             (None, 30)                1530      
                                                                 
 dense_3 (Dense)             (None, 50)                1550      
                                                                 
Total params: 3080 (12.03 KB)
Trainable params: 3080 (12.03 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
#compilling and training the second autoencoder using the encoded features from the first encoder
ae2.compile(optimizer='adam', loss='mse')
ae2.fit(encoded1, encoded1, epochs = 50, batch_size = 8, validation_split = 0.1)

Epoch 1/50
76/76 [==============================] - 1s 3ms/step - loss: 1.5454 - val_loss: 0.7970
Epoch 2/50
76/76 [==============================] - 0s 2ms/step - loss: 0.6806 - val_loss: 0.4427
Epoch 3/50
76/76 [==============================] - 0s 2ms/step - loss: 0.3951 - val_loss: 0.2988
Epoch 4/50
76/76 [==============================] - 0s 2ms/step - loss: 0.2691 - val_loss: 0.2177
Epoch 5/50
76/76 [==============================] - 0s 2ms/step - loss: 0.2003 - val_loss: 0.1681
Epoch 6/50
76/76 [==============================] - 0s 2ms/step - loss: 0.1570 - val_loss: 0.1333
Epoch 7/50
76/76 [==============================] - 0s 2ms/step - loss: 0.1272 - val_loss: 0.1089
Epoch 8/50
76/76 [==============================] - 0s 2ms/step - loss: 0.1054 - val_loss: 0.0908
Epoch 9/50
76/76 [==============================] - 0s 2ms/step - loss: 0.0892 - val_loss: 0.0786
Epoch 10/50
76/76 [==============================] - 0s 2ms/step - loss: 0.0775 - val_loss: 0.0683
Epoch 11/50
76/76 [

## Encoding the first-level features with the second-level encoder

In [15]:
#obtaining the second level of encoded features using the second encoder
encoded2 = encoder2.predict(encoded1)

21/21 [==============================] - 0s 1ms/step


## Construction and training of the third-level autoencoder

In [16]:
#constructing the third Autoencoder

#input layer 
in_ae3 = keras.Input(condensed_e2)
#hidden encoded layer
h_ae3 = keras.layers.Dense(condensed_e3)(in_ae3)
#output layer
out_ae3 = keras.layers.Dense(condensed_e2)(h_ae3)

ae3 = keras.Model(in_ae3, out_ae3)
encoder3 = keras.Model(in_ae3, h_ae3)

ae3.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 30)]              0         
                                                                 
 dense_4 (Dense)             (None, 10)                310       
                                                                 
 dense_5 (Dense)             (None, 30)                330       
                                                                 
Total params: 640 (2.50 KB)
Trainable params: 640 (2.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [17]:
#compilling and training the third autoencoder using the encoded features from the second encoder
ae3.compile(optimizer='adam', loss='mse')
ae3.fit(encoded2, encoded2, epochs = 50, batch_size = 8, validation_split = 0.1)

Epoch 1/50
76/76 [==============================] - 1s 3ms/step - loss: 2.9634 - val_loss: 2.0508
Epoch 2/50
76/76 [==============================] - 0s 2ms/step - loss: 1.9371 - val_loss: 1.4378
Epoch 3/50
76/76 [==============================] - 0s 2ms/step - loss: 1.3860 - val_loss: 1.0655
Epoch 4/50
76/76 [==============================] - 0s 2ms/step - loss: 1.0697 - val_loss: 0.8773
Epoch 5/50
76/76 [==============================] - 0s 2ms/step - loss: 0.8973 - val_loss: 0.7657
Epoch 6/50
76/76 [==============================] - 0s 2ms/step - loss: 0.7824 - val_loss: 0.6811
Epoch 7/50
76/76 [==============================] - 0s 2ms/step - loss: 0.6939 - val_loss: 0.6094
Epoch 8/50
76/76 [==============================] - 0s 2ms/step - loss: 0.6218 - val_loss: 0.5525
Epoch 9/50
76/76 [==============================] - 0s 2ms/step - loss: 0.5620 - val_loss: 0.5022
Epoch 10/50
76/76 [==============================] - 0s 2ms/step - loss: 0.5144 - val_loss: 0.4610
Epoch 11/50
76/76 [

## Constructing the final stacked autoencoder 

In [18]:
#constructing the stacked autoencoder. Initialising all the layers

#input layer which number of neurons equals the number of original features
l_in = keras.Input(features.shape[1])

#hidden layer of encoder 1
l_e1 = keras.layers.Dense(condensed_e1)(l_in)

#hidden layer of encoder 2
l_e2 = keras.layers.Dense(condensed_e2)(l_e1)

#hidden layer of encoder 3
l_e3 = keras.layers.Dense(condensed_e3)(l_e2)

#hidden layer of decoder 2
l_d2 = keras.layers.Dense(condensed_e2)(l_e3)

#hidden layer of decoder 1
l_d1 = keras.layers.Dense(condensed_e1)(l_d2)

#output layer which is the same as the input
l_out = keras.layers.Dense(features.shape[1])(l_d1)

#defining the autoencode
stacked_ae = keras.Model(l_in, l_out)

stacked_ae.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 66)]              0         
                                                                 
 dense_6 (Dense)             (None, 50)                3350      
                                                                 
 dense_7 (Dense)             (None, 30)                1530      
                                                                 
 dense_8 (Dense)             (None, 10)                310       
                                                                 
 dense_9 (Dense)             (None, 30)                330       
                                                                 
 dense_10 (Dense)            (None, 50)                1550      
                                                                 
 dense_11 (Dense)            (None, 66)                3366

## Transferring the trained weights of the 3 autoencoders to the stacked autoencoder

In [19]:
#setting the weights of the stacked autoencoder to those in the trained autoencoders 

#first encoder layer
stacked_ae.layers[1].set_weights(ae1.layers[1].get_weights())
#second encoder layer
stacked_ae.layers[2].set_weights(ae2.layers[1].get_weights())
#third encoder layer
stacked_ae.layers[3].set_weights(ae3.layers[1].get_weights())
#first decoder layer
stacked_ae.layers[4].set_weights(ae3.layers[2].get_weights())
#second decoder layer
stacked_ae.layers[5].set_weights(ae2.layers[2].get_weights())
#second decoder layer
stacked_ae.layers[6].set_weights(ae1.layers[2].get_weights())

## Evaluating the performance of stacked autoencoder

In [20]:
from sklearn.metrics import mean_absolute_error

pred = stacked_ae.predict(X_test)
print(mean_absolute_error(X_test,pred))

6/6 [==============================] - 0s 1ms/step
0.3070091387501804
